In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Dec  2 13:21:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    37W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#from google.colab import drive
#drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#% cd "/content/drive/My Drive/Colab Notebooks/544project/"
#! pwd

In [ ]:
#! ls "/content/drive/My Drive/Colab Notebooks/544project/"

In [5]:
! python --version

Python 3.7.12


In [ ]:
# install environment
! pip3 install transformers

In [7]:
import os
import sys
import math
import numpy as np
import pandas as pd
import time
import scipy.stats
import random

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from transformers import BertTokenizer, BertModel, BertConfig

torch.backends.cudnn.deterministic = True
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(torch.cuda.get_device_name(torch.cuda.current_device()))
print(torch.__version__)
print("Device: {}".format(device))

Tesla P100-PCIE-16GB
1.10.0+cu111
Device: cuda


In [9]:
output_option = ['cls', 'pooler']
output_way = output_option[0]
pretrained_bert = ['bert-base-uncased', 'bert-large-uncased']
pre_embedding = pretrained_bert[0]

In [10]:
learning_rate = 1e-5
batch_size = 64
maxlen = 64
dropouts = 0.15
dup_rate = 0.32
save_path = "./model_saved/best_model13.pth"
train_file = "data/train.csv"
dev_file = "data/sts-dev.csv"
test_file = "data/sts-test.csv"
trainwiki_file = "data/wiki1m_for_simcse.txt"

In [ ]:
# load and test base bert pretrained model
tokenizer = BertTokenizer.from_pretrained(pre_embedding)
Config = BertConfig.from_pretrained(pre_embedding)
model = BertModel.from_pretrained(pre_embedding)
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
Config = BertConfig.from_pretrained(pre_embedding)
Config.attention_probs_dropout_prob = dropouts
Config.hidden_dropout_prob = dropouts

In [12]:
# load the pretrain tokenizer for word repetition
from tokenizers import BertWordPieceTokenizer
repetokenizer = BertWordPieceTokenizer("data/bert-base-uncased-vocab.txt", lowercase=True)

## Data

In [13]:
# load train dataset from QQP
df = pd.read_csv(train_file)
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [14]:
# get data in [Q1_t, Q2_t, duplicate_t]
def get_pairs(df, cols):
    raw = df[cols].values.tolist()
    # delete repeat sentence
    data = []
    count = 0
    for i in range(len(raw)):
        id1 = raw[i][0]
        id2 = raw[i][1]
        q1 = raw[i][2]
        q2 = raw[i][3]
        #label = raw[i][4]

        # check if repeat
        if id1 > count:
            data.append(str(q1))
            count += 1
        if id2 > count:
            data.append(str(q2))
            count += 1

    random.shuffle(data)
    return data

data = get_pairs(df, ['qid1', 'qid2', 'question1', 'question2'])

# mini train
#train_unsup = data[:40000]
# full train
train_unsup = data
print(len(train_unsup))

537933


In [15]:
# word repetition -- ablation
# based on sentence token size to choose the repeat word
def repeset(tokens, dup_rate):
    dup_maxlen = max(2, int(dup_rate*len(tokens)))
    dup_len = random.randint(0, dup_maxlen)
    dup_set = np.random.randint(0, len(tokens)-1, size=dup_len)
    return np.sort(dup_set, kind='mergesort')


def copyword(tokens, dup_set):
    if len(dup_set) == 0:
        return ' '.join(token for token in tokens)
    
    for i in range(len(dup_set)):
        word = tokens[dup_set[i]+i]
        tokens.insert(dup_set[i]+i, word)
        
    return ' '.join(token for token in tokens)


def repetition(data, pre_tokenizer, dup_rate):
    # get tokenized sentence list
    tokenized = []
    for i in range(int(len(data)/2)):
        tokenized += pre_tokenizer.encode_batch([data[i*2], data[i*2+1]])
        
    if len(data)%2 == 1:
        tokenized.append(pre_tokenizer.encode(data[-1]))
        
    # unuseful data which contain only one token
    single = []

    # word repetition through sentence
    dup_sentences = []
    for j in range(len(tokenized)):
        ts = tokenized[j].tokens[1:-1]
        if len(ts) < 2:
            single.append(j)            
        else:
            d_set = repeset(ts, dup_rate)
            dup_sentences.append(copyword(ts, d_set))
    return dup_sentences, single

# remove single word sentence
def train_clean(data, singlewords):
    train_undup = data
    for i in range(len(singlewords)):
        del train_undup[singlewords[i]-i]
    return train_undup

# stack 2 list
def combinedata(o, d):
   output = []
   for i in range(len(o)):
      output.append([o[i], d[i]])
   return output

dup_data, singlewords = repetition(data, repetokenizer, dup_rate)
origin_data = train_clean(data, singlewords)
train_unsup = combinedata(origin_data, dup_data)

In [16]:
# load dev and test set from STS benchmark
def stsbprocess(path):
    stsdata = []
    df = pd.read_csv(path, warn_bad_lines=False, error_bad_lines=False)
    for i in range(len(df.index)):
        item = df.iloc[i][0].split('\t')
        stsdata.append([item[5], item[6], item[4]])
    
    return stsdata

dev_data = stsbprocess(dev_file)
test_data = stsbprocess(test_file)

In [17]:
# load 1 million random wiki sentence for comparsion -- same as simCSE unsup
def wiki1m(path):
    train = []
    with open(path, encoding='utf-8') as f:
        train = f.readlines()  
    random.shuffle(train)
    data = [str(x[:-1]) for x in train]
    return data

#train_unsup = wiki1m(trainwiki_file)

## Model setting

In [18]:
class TrainDataset(Dataset):
    def __init__(self, data, tokenizer, maxlen, transform=None, target_transform=None):
        self.data = data
        self.tokenizer = tokenizer
        self.maxlen = maxlen
        self.transform = transform
        self.target_transform = target_transform

    def text_to_id(self, source):
        #sample = self.tokenizer([source, source], max_length=self.maxlen, truncation=True, padding='max_length', return_tensors='pt')   # simCSE
        sample = self.tokenizer(source, max_length=self.maxlen, truncation=True, padding='max_length', return_tensors='pt')   # word repetition, EsimCSE
        return sample

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.text_to_id(self.data[idx])

In [19]:
class TestDataset:
    def __init__(self, data, tokenizer, maxlen):
        self.tokenizer = tokenizer
        self.maxlen = maxlen
        self.traget_idxs = self.text_to_id([x[0] for x in data])
        self.source_idxs = self.text_to_id([x[1] for x in data])
        self.label_list = [float(x[2]) for x in data]
        assert len(self.traget_idxs['input_ids']) == len(self.source_idxs['input_ids'])

    def text_to_id(self,source):
        sample = self.tokenizer(source, max_length=self.maxlen, truncation=True, padding='max_length', return_tensors='pt')
        return sample

    def get_data(self):
        return self.traget_idxs,self.source_idxs,self.label_list

In [20]:
# build net to do contrastive process
class NeuralNetwork(nn.Module):
    def __init__(self, model_path,output_way):
        super(NeuralNetwork, self).__init__()
        self.bert = BertModel.from_pretrained(model_path, config=Config)
        self.output_way = output_way
        
    def forward(self, input_ids, attention_mask, token_type_ids):
        x1 = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        if self.output_way == 'cls':   # use [cls] by default
            output = x1.last_hidden_state[:,0]
        elif self.output_way == 'pooler':
            output = x1.pooler_output
        return output

In [ ]:
# initialize model
model = NeuralNetwork(pre_embedding, output_way).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [22]:
# data loading
training_data = TrainDataset(train_unsup, tokenizer, maxlen)
train_dataloader = DataLoader(training_data, batch_size=batch_size)

testing_data = TestDataset(test_data, tokenizer, maxlen)
deving_data = TestDataset(dev_data, tokenizer, maxlen)

In [23]:
# Spearman
def compute_corrcoef(x, y):
    return scipy.stats.spearmanr(x, y).correlation

In [24]:
# infoNCE
def compute_loss(y_pred, temp=0.05):
    idxs = torch.arange(0, y_pred.shape[0], device='cuda')   # get id matrix for x and x+, shape[0]==batchsize*2
    y_true = idxs + 1 - idxs % 2 * 2   # get positive pair id matrix
    similarities = F.cosine_similarity(y_pred.unsqueeze(1), y_pred.unsqueeze(0), dim=2)
    #print(torch.eye(y_pred.shape[0], device='cuda'))
    # except itself, 2n-1 negtive pair, n=batch_size
    similarities = similarities-torch.eye(y_pred.shape[0], device='cuda') * 1e12   # exclude itself
    similarities = similarities / temp
    loss = F.cross_entropy(similarities, y_true)
    #print(loss)
    #print(torch.mean(loss))
    return torch.mean(loss)

In [25]:
# DCL without weighting -- ablation
def compute_loss_dcl(y_pred, temp=0.05):
    # split the tensor into source and argumented set
    idxs1 = torch.tensor([i1*2 for i1 in list(range(int(y_pred.shape[0]/2)))], device='cuda')
    idxs2 = torch.tensor([i2*2+1 for i2 in list(range(int(y_pred.shape[0]/2)))], device='cuda')
    y_pred_1 = torch.index_select(y_pred, 0, idxs1)
    y_pred_2 = torch.index_select(y_pred, 0, idxs2)

    # positive pair
    pair_matrix = torch.mm(y_pred_1, y_pred_2.t())
    pos_loss = -torch.diag(pair_matrix) / temp

    # sum of negative pair
    neg_similarity = torch.cat((torch.mm(y_pred_1, y_pred_1.t()), pair_matrix), dim=1) / temp
    neg_mask = torch.eye(y_pred_1.shape[0], device='cuda').repeat(1, 2)
    neg_loss = torch.logsumexp(neg_similarity + neg_mask * np.log(1e-45), dim=1, keepdim=False)
    #print(pos_loss)
    #print(neg_loss)
    return torch.mean(pos_loss + neg_loss)

## Train and eval

In [26]:
# update based on infoNCE/DCL
def train(dataloader, testdata, model, optimizer):
    model.train()
    size = len(dataloader.dataset)
    max_corrcoef = 0
    for batch, data in enumerate(dataloader):
        input_ids = data['input_ids'].view(len(data['input_ids'])*2, -1).to(device)
        attention_mask = data['attention_mask'].view(len(data['attention_mask'])*2, -1).to(device)
        token_type_ids = data['token_type_ids'].view(len(data['token_type_ids'])*2, -1).to(device)
        pred = model(input_ids, attention_mask, token_type_ids)
        loss = compute_loss(pred)
        #print(loss)
        #break
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 10 == 0:
            loss, current = loss.item(), batch * int(len(input_ids)/2)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            model.eval()
            corrcoef = test(testdata, model)
            model.train()
            print(f"corrcoef_test: {corrcoef:>4f}")
            if corrcoef > max_corrcoef:
                max_corrcoef = corrcoef
                torch.save(model.state_dict(), save_path)
                print(f"Higher corrcoef: {(max_corrcoef):>4f}, Saved PyTorch Model State to model.pth")

In [27]:
def test(test_data, model):
    target_idxs, source_idxs, label_list = test_data.get_data()
    with torch.no_grad():
        target_input_ids = target_idxs['input_ids'].to(device)
        target_attention_mask = target_idxs['attention_mask'].to(device)
        target_token_type_ids = target_idxs['token_type_ids'].to(device)
        target_pred = model(target_input_ids, target_attention_mask, target_token_type_ids)

        source_input_ids = source_idxs['input_ids'].to(device)
        source_attention_mask = source_idxs['attention_mask'].to(device)
        source_token_type_ids = source_idxs['token_type_ids'].to(device)
        source_pred = model(source_input_ids, source_attention_mask, source_token_type_ids)
       
        similarity_list = F.cosine_similarity(target_pred, source_pred)
        similarity_list = similarity_list.cpu().numpy()
        label_list = np.array(label_list)
        corrcoef = compute_corrcoef(label_list, similarity_list)
    return corrcoef

In [ ]:
# train
if __name__ == '__main__':
    epochs = 1
    for t in range(epochs):
        print(f"Epoch {t + 1}\n-------------------------------")
        train(train_dataloader, deving_data, model, optimizer)
    print("Train_Done!")

In [29]:
# test
#save_path = "./model_saved/best_model2.pth"
model.load_state_dict(torch.load(save_path))
corrcoef = test(testing_data, model)
print(f"test_corrcoef: {corrcoef:>4f}")

test_corrcoef: 0.739678


simCSE -- batch64 dropout0.1 dev0.825  on 1 million \\
model -- batch64 dropout0.3 dev0.780 \\
model1 -- batch64 dropout0.2 dev0.782 test0.75 \\
model2 -- batch64 dropout0.25 dev0.788 test0.71 \\
model3 -- batch64 dropout0.15 dev0.771 test0.73 \\
model4 -- batch64 dropout0.25 decouple dev0.751 temp0.05 \\
model5 -- batch64 dropout0.15 decouple dev0.76 temp0.05 \\
model6 -- batch64 dropout0.2 decouple dev0.754 temp0.05\\
model7 -- batch64 dropout0.15 decouple dev0.773 temp0.2 \\
model8 -- batch64 dropout0.1 wiki1m \\
model9 -- batch64 dropout0.1 wordrepetition dev0.783 test0.757 \\
model10 -- batch64 dropout0.1 dev0.744 \\
model11 -- batch64 dropout0.2 wordrepetition dev0.793 test0.74 \\
model12 -- batch64 dropout0.25 wordrepetition dev790 test0.72 \\
model13 -- batch64 dropout0.15 wordrepetition dev0.784 test0.74 \\